In [71]:
import pinecone
from pinecone import Pinecone
from langchain_openai import ChatOpenAI
from pipeline.embeddings import HuggingFaceEmbeddings
from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnableSequence, RunnableLambda
from langchain_pinecone import PineconeVectorStore
import re
import os
from dotenv import load_dotenv
from pipeline.utils import convert_bold_to_html

load_dotenv()
PINECONE_API = os.getenv("PINECONE_API_KEY")
OPENAI_API = os.getenv("OPENAI_API_KEY")
# HUGGINGFACE_API = os.getenv("HUGGINGFACE_API_KEY")

# Initialize Pinecone
pinecone = Pinecone(api_key=PINECONE_API)
chunk_index = pinecone.Index("education-file-chunks")
file_info_index = pinecone.Index("education-file-info")

# Use your custom embeddings class
embedding_model = HuggingFaceEmbeddings()

# Create a vector store for chunks
chunk_vectorstore = PineconeVectorStore(
    index=chunk_index, 
    embedding=embedding_model, 
    text_key='text'
)

# Create a vector store for file info (summaries)
file_info_vectorstore = PineconeVectorStore(
    index=file_info_index, 
    embedding=embedding_model, 
    text_key='id'
)

# Chain to use the LLM with the prompt
llm_for_query = ChatOpenAI(model_name="gpt-4o-mini", 
                           openai_api_key=OPENAI_API)
llm_for_response = ChatOpenAI(model_name="gpt-4o-mini", 
                              openai_api_key=OPENAI_API)

def query_vectorstore(vectorstore: PineconeVectorStore, user_query: str, top_k: int = 3, **kwargs):
    # Step 1: Query Pinecone
    results = vectorstore.similarity_search(user_query, k=top_k, **kwargs)
    if not results:
        return "No relevant results found."

    return results

def query_for_file(user_query: str, top_k: int = 3):
    """
    Query the vector store for file info (summaries) and return the top_k file_ids.
    """
    query_res = query_vectorstore(file_info_vectorstore, user_query, top_k)

    file_ids = [doc.page_content for doc in query_res]
    return file_ids

def query_for_chunks(user_query: str, file_ids: list[str], top_k: int = 3):
    """
    Given the target file_ids, query the vector store for top_k chunks FOR EACH FILE.
    """

    # Pass a filter to restrict the search to the identified files.
    final_chunks = []
    for file_id in file_ids:
        query_res = query_vectorstore(chunk_vectorstore, user_query, top_k, filter={"file_path": file_id})
        chunks = [doc.page_content for doc in query_res]
        final_chunks.extend(chunks)
    return final_chunks

# Define the prompt template
query_prompt_template = PromptTemplate(
    input_variables=["user_query"],
    template=(
        "You are an advanced query assistant with expertise in carbon credits. "
        "Analyze the user's input to construct a meaningful, contextually complete query that can be vectorized for semantic similarity search. "
        "For clarity, you can break the input into clear and meaningful components for easy similarity search. "
        "Ensure the query is well-structured, includes all relevant information needed, and is cleaned of special characters and converted to lowercase. \n\n"
        "User Input: {user_query}\n"
        "Construct the query in the following format:\n\n"
        "Query: <constructed query>\n"
    )
)

# Define a prompt for response generation
response_generation_prompt = PromptTemplate(
    input_variables=["context", "user_query"],
    template=(
        "You are an expert with strong expertise in carbon credits. Based on the following context, respond to the user's query. Try to give a very long response that is as detailed as possible, with lots of information. And remember to be factual:\n\n"
        "Context: {context}\n\n"
        "User Query: {user_query}\n\n"
        "Response:"
    )
)

def debug_step(name):
    """Debug function to print the state of variables."""
    return RunnableLambda(func=lambda inputs: {**inputs, "debug": print(f"{name}: {inputs}")})

# Chain step 1: Retrieve file IDs (5 most relevant files)
file_query_chain = RunnableLambda(
    func=lambda inputs: {
        "file_ids": query_for_file(
            user_query=inputs["user_query"],
            top_k=5  # Query 5 most relevant files
        ),
        "user_query": inputs["user_query"]
    }
)

# Chain step 2: For the retrieved file IDs, query for chunks (3 best matching chunks per file)
chunks_query_chain = RunnableLambda(
    func=lambda inputs: {
        "chunks": query_for_chunks(
            user_query=inputs["user_query"],
            file_ids=inputs["file_ids"],
            top_k=3  # Query 3 best-matching chunks per file
        ),
        "user_query": inputs["user_query"],
        "consulted_files": [file_id.removeprefix("/content/drive/MyDrive/") for file_id in inputs["file_ids"]]
    }
)

# Chain step 3: Aggregate the chunks to form the context
aggregation_chain = RunnableLambda(
    func=lambda inputs: {
        "context": "\n".join(inputs["chunks"]),
        "user_query": inputs["user_query"],
        "consulted_files": inputs["consulted_files"]
    }
)

# Chain step 4: Generate the final response using the aggregated context and the original user query
response_chain = RunnableLambda(
    func=lambda inputs: {
        "response_prompt": response_generation_prompt.format(
            context=inputs["context"],
            user_query=inputs["user_query"]
        ),
        "consulted_files": inputs["consulted_files"]

    }
) | RunnableLambda(
    func=lambda inputs: {
        "response": llm_for_response.predict(
            response_prompt=inputs["response_prompt"]
        ),
        **inputs
    }
)

# Combine all the chains to form the full workflow
full_chain = file_query_chain | chunks_query_chain | aggregation_chain | response_chain
print("First intialization")

def generate_response(user_query: str):
    # Run the full chain
    result = full_chain.invoke(input={"user_query": user_query})
    response = result["response"]
    consulted_files = result["consulted_files"]
    final_response = response + f"\n\n Nguồn thông tin: {'\n- '.join(consulted_files)}"
    return final_response

In [69]:
user_query = "quản lý cải thiện rừng"

file_ids = query_for_file(user_query, top_k=5)
file_ids

['/content/drive/MyDrive/Tai_lieu__1001_ieu_ban_can_biet_ve_Tin_chi_Carbon_Revised/5._Phan_5__nature_based_carbon_credit_project/15._Chapter_15__Improved_Forest_Management_(IFM)/Revised_VFCS_ST_1003_2019_Final_draft_for_public_consensus.pdf',
 '/content/drive/MyDrive/Tai_lieu__1001_ieu_ban_can_biet_ve_Tin_chi_Carbon_Revised/5._Phan_5__nature_based_carbon_credit_project/13._Chapter_13__Trong_rung_va_tai_trong_rung__Afforestation,_Reforestation,_and_Revegetation_(ARR)/OP_237.pdf',
 '/content/drive/MyDrive/Tai_lieu__1001_ieu_ban_can_biet_ve_Tin_chi_Carbon_Revised/2._Phan_2__Tin_Chi_Carbon_La_gi_/5._Chuong_5__Chinh_sach_phat_trien_thi_truong_tin_chi_carbon_tai_Viet_Nam/Luat_lien_quan/Decree_No._1562018ND_CP_(amended_and_supplemented)_Vietnamese_Version.pdf',
 '/content/drive/MyDrive/Tai_lieu__1001_ieu_ban_can_biet_ve_Tin_chi_Carbon_Revised/5._Phan_5__nature_based_carbon_credit_project/16._Chapter_16__Reduced_Emisstions_from_Deforestation_and_Degradation_(REDD+)/2011chuyendongcungredd.pdf',

In [70]:
chunks = query_for_chunks(user_query, file_ids, top_k=1)
chunks

[['/content/drive/MyDrive/Tai_lieu__1001_ieu_ban_can_biet_ve_Tin_chi_Carbon_Revised/5._Phan_5__nature_based_carbon_credit_project/15._Chapter_15__Improved_Forest_Management_(IFM)/Revised_VFCS_ST_1003_2019_Final_draft_for_public_consensus.pdf',
  'ch ủ rừng ph ải xây d ựng và b ảo trì các công trình h ạ tầng k ỹ thuật phù hợp v ới m ục tiêu quản lý và h ạn ch ế ảnh hư ởng môi trường 4 8 1 lsng có k ế hoạch và thực hiện xây d ựng và bảo trì đường c ầu cống đư ờng vận xuất vận chuy ển và bãi t ập kết lâm s ản ngoài g ỗ các công trình đư ợc thể hiện trên b ản đồ không b ă t buộc đối với chủ rừng là hộ gia đ ình c á nhân v à cộng đồng dân cư 4 8 2 lsng vi ệc xây d ựng và bảo trì đư ờng c ầu cống đường vận xuất vận chuy ển và bãi t ập kết lâm s ản ngoài g ỗ đảm bảo không gây tác đ ộng tiêu c ực tới môi trư ờng nguyên t ắc 5 qu ản lý và b ảo v ệ môi trư ờng trong các hoạt động lâm nghi ệp 5 1 1 lsng đ ánh gi á các ảnh hư ởng tiêu c ực có th ể xảy ra đối với môi trư ờng của các ho ạt động qu ả

In [60]:
results = chunk_vectorstore.similarity_search(user_query, k=5, filter={"file_path": file[]})
results

[Document(id='/content/drive/MyDrive/Tai_lieu__1001_ieu_ban_can_biet_ve_Tin_chi_Carbon_Revised/5._Phan_5__nature_based_carbon_credit_project/15._Chapter_15__Improved_Forest_Management_(IFM)/Revised_VFCS_ST_1003_2019_Final_draft_for_public_consensus.pdf_chunk_71', metadata={'chunk_id': 71.0, 'file_path': '/content/drive/MyDrive/Tai_lieu__1001_ieu_ban_can_biet_ve_Tin_chi_Carbon_Revised/5._Phan_5__nature_based_carbon_credit_project/15._Chapter_15__Improved_Forest_Management_(IFM)/Revised_VFCS_ST_1003_2019_Final_draft_for_public_consensus.pdf'}, page_content='ch ủ rừng ph ải xây d ựng và b ảo trì các công trình h ạ tầng k ỹ thuật phù hợp v ới m ục tiêu quản lý và h ạn ch ế ảnh hư ởng môi trường 4 8 1 lsng có k ế hoạch và thực hiện xây d ựng và bảo trì đường c ầu cống đư ờng vận xuất vận chuy ển và bãi t ập kết lâm s ản ngoài g ỗ các công trình đư ợc thể hiện trên b ản đồ không b ă t buộc đối với chủ rừng là hộ gia đ ình c á nhân v à cộng đồng dân cư 4 8 2 lsng vi ệc xây d ựng và bảo trì đư